In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [2]'.</span>

An Exception was encountered at 'In [2]'.

# Working with Stata

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
  * stata does not have a concept of variable and can only exchange stata dataset as dataframe  

## Stata <a id="Stata"></a>

Stata, similar to SAS, has one and only one **unnamed current dataset** in memory. It also supports **named, untyped macros**. SoS supports the exchange of variables between Stata and other kernels as follows:

When pushing data to `Stata`

| Python  | variable name | Stata |
| --- | ---| ---|
| `int`, `float`, `str` | variable name become macro name |  local macro (`string`) |
| `pandas.DataFrame` | unnamed in Stata | Stata Dataset |


When getting data from `Stata`

| Stata  | variable name | Type in Python |
| --- | --- |  ---|
| local macros | name of macro | `str` |
| global macros | name of macro | `str` |
| Dataset  | user specified | `pandas.DataFrame` |

Note that non-dataframe variables are transferred to Stata as local macros, and both local and global macros can be transferred although local macros will be checked first.

For example, if you have a directory and filenames defined in SoS

In [1]:
datadir = '~/Downloads/StataIntro/dataSets'
filename = 'gss.dta'

You can use the variables instantly in Stata with SoS' own `%expand` magic,

In [2]:
%expand
cd {datadir}
use {filename}
ds

ValueError: No subkernel named Stata is found. Please make sure that you have the kernel installed (listed in the output of "jupyter kernelspec list" and usable in jupyter by itself), install appropriate language module (e.g. "pip install sos-r"), restart jupyter notebook and try again.

However, if you prefer, you can transferred the variables as local macros in Stata

In [ ]:
%get datadir filename
macro list

and use the macros in Stata

In [ ]:
cd `datadir'
use `filename'
ds

Now if you have a dataframe, for example, `mtcard` in R, getting them in Stata will **replace the current dataset**, and the name `mtcars` no longer matters.

In [ ]:
%get mtcars --from R
summarize

In [ ]:
hist hp

Sending macros and/or dataset to another kernel is a bit tricky because local and global macros can have the same name, and the dataset is unnamed. The rule is that

1. If the name represents a local macro, send it as string
2. Otherwise if the name represents a global macro, send it as string
3. Otherwise send the current dataset as a DataFrame using the provided name

For example, because `mydata` is not one of the macros, it will become the name of the DataFrame representing the Stata dataset:

In [ ]:
%get mydata --from Stata
mydata

The macros can be obtained 

In [ ]:
%get S_ADO --from Stata
S_ADO

and Stata supports the automatic transfer of variables (macros) with names starting with `sos`. For example,

In [ ]:
local sosres 3.14159

but remember macros are untyped so they will be transferred as strings:

In [ ]:
sosres

The `%sessioninfo` magic lists the version of Stata when the `stata` kernel is used:

## Further reading

* [How to exchange variables among living subkernels](doc/user_guide/exchange_variable.html)